In [1]:
!ls -lh data/

total 842M
-rw-r--r-- 1 RENT 197121 459M ׳“׳¦׳� 11 15:34 fixed.csv
-rw-r--r-- 1 RENT 197121 379M ׳“׳¦׳� 11 14:18 oddetect.csv
-rw-r--r-- 1 RENT 197121 4.1M ׳“׳¦׳� 12 10:01 paths.pkl.xz
-rw-r--r-- 1 RENT 197121 368K ׳“׳¦׳� 12 09:10 paths0.png
-rw-r--r-- 1 RENT 197121 2.5K ׳“׳¦׳� 12 00:13 sample.csv


# Loading The CSV File

In [21]:
%%time

import pandas as pd
df = pd.read_csv('data/fixed.csv')

Wall time: 20.7 s


In [22]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997978 entries, 0 to 2997977
Data columns (total 14 columns):
0                                     int64
 14                                   int64
 329                                  int64
 0                                    int64
 362                                  int64
 0.1                                  int64
 0.2                                  int64
 0.3                                  int64
 5                                    int64
 14-16\20170817012709_100.h264.txt    object
 2017-08-17 01:27:09                  object
 01:27:09                             object
 0:00:00                              object
 0.0604166666667                      float64
dtypes: float64(1), int64(9), object(4)
memory usage: 1.1 GB


In [23]:
len(df)

2997978

In [4]:
df.describe()

,0,14,329,0,362,0.1,0.2,0.3,5,0.0604166666667
count,2.997978e+06,2.997978e+06,2.997978e+06,2.997978e+06,2.997978e+06,2.997978e+06,2997978.0,2997978.0,2.997978e+06,2.997978e+06
mean,8.289600e+03,2.730163e+02,1.480890e+02,5.053652e+01,4.675814e+07,1.158957e+02,0.0,0.0,7.658547e+00,5.179974e-01
std,5.189148e+03,1.609973e+02,7.650768e+01,8.449937e+01,3.391169e+07,1.502103e+02,0.0,0.0,1.366049e+01,2.580195e-01
min,0.000000e+00,3.000000e+00,3.000000e+00,0.000000e+00,3.620000e+02,0.000000e+00,0.0,0.0,1.000000e+00,0.000000e+00
25%,3.712000e+03,1.440000e+02,9.500000e+01,1.200000e+01,1.419073e+07,1.600000e+01,0.0,0.0,2.000000e+00,3.541667e-01
50%,8.119000e+03,2.690000e+02,1.340000e+02,2.800000e+01,4.510348e+07,5.900000e+01,0.0,0.0,3.000000e+00,5.277778e-01
75%,1.263600e+04,4.020000e+02,2.050000e+02,5.100000e+01,5.399656e+07,1.620000e+02,0.0,0.0,6.000000e+00,7.138889e-01
max,1.847600e+04,6.370000e+02,3.570000e+02,6.430000e+02,1.265195e+08,1.977000e+03,0.0,0.0,1.150000e+02,9.993056e-01


In [5]:
df.columns = ['frame', 'x', 'y', 'obj', 'size', 'seq', 'tbd1', 'tbd2','tbd3', 'filename', 'start', 'path_time', 'delta_time','tbd4']

In [6]:
df.head()

,frame,x,y,obj,size,seq,tbd1,tbd2,tbd3,filename,start,path_time,delta_time,tbd4
0,0,4,291,1,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
1,0,629,7,2,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
2,0,253,4,3,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
3,0,185,4,4,362,0,0,0,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00,0.060417
4,1700,5,102,5,362,0,0,0,1,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:28:17,0:01:08,0.061111


In [7]:
%%time
import pandas as pd
df = pd.read_csv('data/fixed.csv',usecols=[1,2,3,5,9,10])
df.columns = ['x', 'y', 'obj', 'seq', 'filename', 'time']


Wall time: 8.37 s


In [8]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997978 entries, 0 to 2997977
Data columns (total 6 columns):
x           int64
y           int64
obj         int64
seq         int64
filename    object
time        object
dtypes: int64(4), object(2)
memory usage: 571.7 MB


In [9]:
df['time'] = pd.to_datetime(df['time'])

In [10]:
%%time

df_int = df.select_dtypes(include=['int64'])
converted_int = df_int.apply(pd.to_numeric,downcast='unsigned')
                             
df[converted_int.columns] = converted_int
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997978 entries, 0 to 2997977
Data columns (total 6 columns):
x           uint16
y           uint16
obj         uint16
seq         uint16
filename    object
time        datetime64[ns]
dtypes: datetime64[ns](1), object(1), uint16(4)
memory usage: 305.8 MB
Wall time: 2.49 s


In [11]:
%%time
df_obj = df.select_dtypes(include = ['object']).copy()
converted_obj = pd.DataFrame()

for col in df_obj.columns:
       num_unique_values = len(df_obj[col].unique())
       num_total_values = len(df_obj[col])
       if num_unique_values / num_total_values < 0.5:
           converted_obj.loc[:,col] = df_obj[col].astype('category')
       else:
           converted_obj.loc[:,col] = df_obj[col]

df[converted_obj.columns] = converted_obj

Wall time: 1.15 s


In [12]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2997978 entries, 0 to 2997977
Data columns (total 6 columns):
x           uint16
y           uint16
obj         uint16
seq         uint16
filename    category
time        datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint16(4)
memory usage: 51.5 MB


In [13]:
df

,x,y,obj,seq,filename,time
0,4,291,1,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
1,629,7,2,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
2,253,4,3,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
3,185,4,4,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
4,5,102,5,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
5,6,102,5,1,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
6,6,102,5,2,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
7,7,102,5,3,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
8,9,102,5,4,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09
9,11,102,5,5,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09


In [19]:
df = df.drop_duplicates()

In [20]:
len(df)

1779632

In [16]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1779632 entries, 0 to 2997818
Data columns (total 6 columns):
x           uint16
y           uint16
obj         uint16
seq         uint16
filename    category
time        datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint16(4)
memory usage: 44.2 MB


In [17]:
%%time
df.to_pickle('data/paths.pkl.xz')

Wall time: 30.4 s


In [18]:
%%time
df2= pd.read_pickle('data/paths.pkl.xz')
df2.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1779632 entries, 0 to 2997818
Data columns (total 6 columns):
x           uint16
y           uint16
obj         uint16
seq         uint16
filename    category
time        datetime64[ns]
dtypes: category(1), datetime64[ns](1), uint16(4)
memory usage: 44.2 MB
Wall time: 2.01 s
